<a href="https://colab.research.google.com/github/antonpetkoff/identifier-suggestion/blob/master/notebooks/00-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# import utilities
import os
import shutil
from subprocess import check_output

In [2]:
!nvidia-smi

Fri May  8 20:30:51 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
%env WORKSPACE_DIR=/content/gdrive/My Drive/src

# TODO: how can one read an environment variable?!?
%cd '/content/gdrive/My Drive/src'

env: WORKSPACE_DIR=/content/gdrive/My Drive/src
/content/gdrive/My Drive/src


In [0]:
timestamp = check_output(['date', '-u', "+%Y-%m-%dT%H-%M-%S"]).decode('utf-8').strip()

os.environ['PROJECT_DIR'] = os.path.join(
    os.environ['WORKSPACE_DIR'],
    f'identifier-suggestion-{timestamp}',
)

In [6]:
!git clone https://github.com/antonpetkoff/identifier-suggestion.git --depth 1 "${PROJECT_DIR}"

Cloning into '/content/gdrive/My Drive/src/identifier-suggestion-2020-05-08T20-31-22'...
remote: Enumerating objects: 62, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 62 (delta 0), reused 38 (delta 0), pack-reused 0
Unpacking objects: 100% (62/62), done.


In [0]:
os.chdir(os.environ['PROJECT_DIR'])

In [8]:
!pwd
!ls -l

/content/gdrive/My Drive/src/identifier-suggestion-2020-05-08T20-31-22
total 54
-rw------- 1 root root 35149 May  8 20:31 LICENSE
drwx------ 2 root root  4096 May  8 20:31 notebooks
-rw------- 1 root root  6902 May  8 20:31 README.md
drwx------ 2 root root  4096 May  8 20:31 requirements
drwx------ 9 root root  4096 May  8 20:31 src


KeyboardInterrupt: ignored

In [9]:
%tensorflow_version 2.x

import tensorflow
print(tensorflow.__version__)

2.2.0-rc4


In [10]:
# Google Colab has standard libraries like numpy, pandas, matplotlib and TF (of course) pre-installed
!pip install -r requirements/colab.txt

     |████████████████████████████████| 1.4MB 6.2MB/s 
     |████████████████████████████████| 4.3MB 16.6MB/s 
     |████████████████████████████████| 112kB 49.7MB/s 
     |████████████████████████████████| 460kB 60.4MB/s 
     |████████████████████████████████| 102kB 14.2MB/s 
     |████████████████████████████████| 102kB 13.1MB/s 
     |████████████████████████████████| 71kB 11.2MB/s 
     |████████████████████████████████| 71kB 10.2MB/s 
  Created wheel for watchdog: filename=watchdog-0.10.2-cp36-none-any.whl size=73605 sha256=e69811a3b60cad312492ab0d28e0c46d7bd91cd5a5d87b9dabad2e4a588365c0
  Stored in directory: /root/.cache/pip/wheels/bc/ed/6c/028dea90d31b359cd2a7c8b0da4db80e41d24a59614154072e
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6489 sha256=3a37143cfb89d4c47be785d401423858bca703bf3b8d433cfd1343055f0f00ab
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for gql

In [11]:
# provide secrets to the project, e.g. access to wandb
shutil.copy(
    os.path.join(os.environ['WORKSPACE_DIR'], 'secrets/.env'),
    os.environ['PROJECT_DIR']
)

'/content/gdrive/My Drive/src/identifier-suggestion-2020-05-08T20-31-22/.env'

In [0]:
# monkey-patch (mock) os.symlink to be a noop, because wandb.save() uses it, but it is not supported by Google Colab Notebooks
os.symlink = lambda *x: print('Executing mocked noop symlink with arguments', x)

In [14]:
from argparse import Namespace
from src.pipelines.baseline import run

params = {
  'file_data_raw': '../data/interim/method-names-rich/elasticsearch.csv',
  'file_model_dir': 'models/saved/baseline/',
  'dir_preprocessed_data': '../data/interim/preprocessed/',
  'max_input_length': 200,
  'max_output_length': 8,
  'input_vocab_size': 20000,
  'input_embedding_dim': 128,
  'output_vocab_size': 6000,
  'output_embedding_dim': 128,
  'latent_dim': 512,
  'learning_rate': 0.0001,
  'epochs': 300,
  'batch_size': 512,
  'random_seed': 1,
}

run(Namespace(**params))

Experiment parameters:  Namespace(batch_size=512, dir_preprocessed_data='../data/interim/preprocessed/', epochs=300, file_data_raw='../data/interim/method-names-rich/elasticsearch.csv', file_model_dir='models/saved/baseline/', input_embedding_dim=128, input_vocab_size=20000, latent_dim=512, learning_rate=0.0001, max_input_length=200, max_output_length=8, output_embedding_dim=128, output_vocab_size=6000, random_seed=1)


Streaming output truncated to the last 5000 lines.
epoch 53 - batch 30 - loss 0.25422248244285583 - precision 0.58203125 - recall 0.58203125 - f1 0.58203125 - sparse_categorical_accuracy 0.5780320167541504
epoch 53 - batch 40 - loss 0.21478749811649323 - precision 0.5651576450892857 - recall 0.5651576450892857 - f1 0.5651576450892857 - sparse_categorical_accuracy 0.5613071918487549
epoch 53 - batch 50 - loss 0.14953406155109406 - precision 0.5444308035714286 - recall 0.5444308035714286 - f1 0.5444308035714286 - sparse_categorical_accuracy 0.5409319400787354
epoch 53 - batch 60 - loss 0.16796255111694336 - precision 0.5281110491071429 - recall 0.5281110491071429 - f1 0.5281110491071429 - sparse_categorical_accuracy 0.5248651504516602
epoch 53 - batch 70 - loss 0.15581731498241425 - precision 0.5170918367346938 - recall 0.5170918367346938 - f1 0.5170918367346938 - sparse_categorical_accuracy 0.5140266418457031
epoch 53 - batch 80 - loss 0.1935911923646927 - precision 0.5076311383928571 -